In [ ]:
# Installing exotic packages:
# %pip install pysolar exifread timezonefinder opencv-python tqdm suncalc scikit-learn pandas

In [ ]:
import numpy as np
import math
import pysolar.solar as solar
import suncalc
import datetime
import exifread
from timezonefinder import TimezoneFinder
import pytz
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1" # required to enable .exr for new opencv-python versions
import cv2
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
%config InlineBackend.print_figure_kwargs = {'facecolor' : "w"} # making matplotlib background white
from pathlib import Path
import sys
from tqdm.auto import tqdm
import multiprocessing as mp
import pandas as pd

## Dataset configuration

https://cgg.mff.cuni.cz/~tobias/sky_dataset.php

### GPS coordinates of the shooting places

the list of shootings (and metadata) is available in the Google Sheet "[SkyGAN] picture shooting log"

- 2019-06-26_0848_prague_chodska_21_rooftop: there is an anomaly at the end - maybe just invalid file timestamps?
- 2019-07-15_1929_prague_chodska_21_rooftop: the Sun goes down way faster than expected - maybe incorrect time on the camera? or bad zoom?

In [ ]:
places_dict = {
    #'place_or_shooting_name': (latitude, longitude),
    'prague_chodska_21': (50.074958, 14.445574),
    '2019-05-24_1451_divoka_sarka': (50.09649, 14.321870),
    '2019-06-04_1010_ronan': (50.0883, 14.4481),
    '2019-06-04_1117_ronan': (50.0999, 14.4640),
    '2019-06-04_1659_ronan': (50.0964, 14.3203),
    '2019-06-05_1215_ronan': (50.0869, 14.4518),
    '2019-06-05_1700_ronan': (50.0964, 14.3203),
    '2019-06-06_1158_ronan': (50.0891, 14.4524),
    '2019-06-06_1257_ronan': (50.0851, 14.4602),
    '_hammerschmiede': (48.944980, 9.972590),
    '2019-06-08_1545_quarry': (49.292050, 9.808800),
    'farm_vystice': (49.122424, 14.390338),
    '2019-06-29_1113_divoka_sarka': (50.1006875, 14.3222442),
    '2019-07-04_1749_lake_lhota': (50.24481, 14.66734),
    'santa_cruz_villa_nuova': (36.976024, -122.020524),
    'burbank_1200_riverside': (34.158442, -118.312839),
    'centrum_nesmen_by_te_wood_logs':  (48.8095625, 14.5562914),
    'centrum_nesmen_by_the_wood_logs': (48.8095625, 14.5562914),
    'centrum_nesmen_hilltop': (48.8131133, 14.5541778),
}

## Functions

### Date-time retrieval

In [ ]:
def get_datetime_from_exif(fname):
    with open(fname, 'rb') as f:
        tags = exifread.process_file(f)
        dt = tags['EXIF DateTimeOriginal'].values
        return datetime.datetime.strptime(dt, "%Y:%m:%d %H:%M:%S")
        # NOTE: EXIF's time zone implementation is only quite recent and not supported here

def get_latlong_from_shooting_name(datetime_and_place):
    matched = None # (place_name, (lat, long))
    for place_name, pos in places_dict.items():
        if datetime_and_place.__contains__(place_name):
            if matched is not None:
                raise Exception('multiple places matched: {} and {}'.format(matched[0], place_name))
            matched = (place_name, pos)
    if matched is None:
        raise Exception('position not found based on directory name: add it to "places_dict"')
    return matched[1]

def get_latlong(fname):
    assert(fname[-3:] == 'CR2')
    datetime_and_place = fname.split('/')[-2]
    # TODO assert format "YYYY-MM-DD_HHMM_placename"
    return get_latlong_from_shooting_name(datetime_and_place)

def get_datetime(fname, tzf): # this one 
    latitude, longitude = get_latlong(fname)
    t = tzf.timezone_at(lng=longitude, lat=latitude)
    tz = pytz.timezone(t)
    dt = get_datetime_from_exif(fname)
    return tz.localize(dt)

### Sun coordinates from location and date-time

In [ ]:
def compute_elevation_azimuth(latitude, longitude, date):
    result = suncalc.get_position(date.astimezone(datetime.timezone.utc),
        longitude, latitude)
    return np.rad2deg(result["altitude"]), np.rad2deg(result["azimuth"]) + 180
    # return (solar.get_altitude(latitude, longitude, date), # solar.get_altitude(): calculate the angle between the sun and a plane tangent to the earth where you are. The result is returned in degrees.
    #         solar.get_azimuth(latitude, longitude, date))

### Sun coordinates from an image (searching for a solar disk in an image)

In [ ]:
def get_elevation_azimuth_from_fisheye_xy(xy, radius):
    x, y = xy

    def dist_sqr(x1, x2, y1, y2):
        return (x1-x2)*(x1-x2) + (y1-y2)*(y1-y2)

    center = radius

    if dist_sqr(center, x, center, y) >= radius * radius: # is inside the dome-circle?
        # raise Exception('Querying a fisheye coordinate outside of the dome-circle')
        return np.nan, np.nan

    x_rel = (x - center) / radius
    y_rel = (y - center) / radius

    # print(x_rel, y_rel)

    xy_rel_len = math.sqrt(x_rel*x_rel + y_rel*y_rel)

    x_rel_norm = x_rel / xy_rel_len
    y_rel_norm = y_rel / xy_rel_len

    azimuth_rad = math.atan2(y_rel_norm, x_rel_norm)
    if azimuth_rad < math.pi:
        azimuth_rad += 2 * math.pi
    
    elevation_rad = 2 * math.atan(1/xy_rel_len) - math.pi / 2.0

    azimuth_picture_deg = azimuth_rad * 180 / math.pi
    elevation_deg = elevation_rad * 180 / math.pi

    # conversion between real-world azimuth and our visualization (fisheye photos of skydome model ... north is up, west is to the right)
    azimuth_realworld_deg = 270 - azimuth_picture_deg
    if azimuth_realworld_deg < 0:
        azimuth_realworld_deg += 360
        
    return elevation_deg, azimuth_realworld_deg

# def get_elevation_azimuth_from_fisheye_xy(xy_imagecoord, radius):
#     x_imagecoord, y_imagecoord = xy_imagecoord

#     def dist_sqr(x1, x2, y1, y2):
#         return (x1-x2)*(x1-x2) + (y1-y2)*(y1-y2)

#     # if x_imagecoord * x_imagecoord + y_imagecoord * y_imagecoord > radius * radius: # is inside the dome-circle?
#     #     raise Exception('Querying a fisheye coordinate outside of the dome-circle')

#     x = (x_imagecoord - radius) / radius
#     y = (y_imagecoord - radius) / radius

#     print(x, y)

#     if x * x + y * y > 1.0: # is inside the dome-circle?
#         raise Exception('Querying a fisheye coordinate outside of the dome-circle')

#     z = np.sqrt(1.0 - x * x - y * y) # assuming the upper hemisphere

#     elevation = np.pi / 2.0 - np.arccos(z)
#     azimuth = np.arctan2(y, x) + np.pi
        
#     return (np.rad2deg(elevation), np.rad2deg(azimuth))

In [ ]:
# def find_solar_disk_center(image):
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
#     mask_threshold = 0.5
#     image_mask = (image >= mask_threshold).astype(float) # 1 = bright pixel, 0 = not bright pixel
#     # plt.imshow(image_mask)
#     image_mask_nonzero = np.transpose(image_mask.nonzero())
#     if len(image_mask_nonzero) > 0:
#         center = np.mean(np.transpose(image_mask.nonzero()), axis=0)[::-1]
#     else:
#         center = [0,0]
#     # plt.scatter(center[0], center[1], marker='x')
#     # plt.xlim(500,700)
#     # plt.ylim(600,800)
#     return center

def find_solar_disk_center(image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    mask_threshold = 0.5
    _, image_mask = cv2.threshold(image, mask_threshold, 255, cv2.THRESH_BINARY)
    image_mask = np.array(image_mask).astype(np.uint8)
    plt.imshow(image_mask)
    plt.show()

    #im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_FLOODFILL, cv2.CHAIN_APPROX_SIMPLE)
    contours, hierarchy = cv2.findContours(image_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    #contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    #print(contours)
    #print(hierarchy.shape)
    # num_contours = hierarchy.shape[1]
    if hierarchy is None or hierarchy.shape[1] != 1:
        return [0,0] # Multiple bright objects found
    
    #print(len(contours[0]))
    if len(contours[0]) < 5:
        return [0,0] # Too few points - probably not an ellipse

    ellipse = cv2.fitEllipse(contours[0])

    return ellipse[0] # ellipse in the format (center x, center y), (width, height), angle in degrees

In [ ]:
# Test:

path = "/projects/SkyGAN/clouds_fisheye/processed/2019-06-24_0504_prague_chodska_21_rooftop/1K_EXR/IMG_9864_hdr.exr"
path = "/projects/SkyGAN/clouds_fisheye/processed/2019-07-14_1418_prague_chodska_21_rooftop/1K_EXR/IMG_3596_hdr.exr"
image = cv2.imread(path, flags = cv2.IMREAD_ANYDEPTH | cv2.IMREAD_COLOR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()
solar_disk_center = find_solar_disk_center(image)
get_elevation_azimuth_from_fisheye_xy(solar_disk_center[::-1], np.shape(image)[0] / 2.0)

## Processing the dataset

In [ ]:
# Given a list of .exr files, it checks their timestamps and tries to detect the solar disk positions in the files
def process_subset(paths_exr):
    tzf = TimezoneFinder() # needs to exist per each thread separately!
    results = {
        "t": [],
        "lat": [],
        "lon": [],
        "detected_azimuth": [],
        "detected_elevation": [],
        "img_fname": [],
    }
    for path_exr in paths_exr:

        path_raw = Path(str(path_exr).replace('/processed/', '/raw/').replace('/1K_EXR', '').replace('_hdr.exr', '.CR2'))
        assert path_raw.exists()

        results["img_fname"].append(str(path_exr))

        latitude, longitude = get_latlong(str(path_raw))
        datetime_ = get_datetime(str(path_raw), tzf)
        results["t"].append(datetime_)
        results["lat"].append(latitude)
        results["lon"].append(longitude)

        image = cv2.imread(str(path_exr), flags = cv2.IMREAD_ANYDEPTH | cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        detected_solar_disk_center = find_solar_disk_center(image)
        detected_elevation, detected_azimuth = get_elevation_azimuth_from_fisheye_xy(
            detected_solar_disk_center, np.shape(image)[0] / 2.0)
        results["detected_elevation"].append(detected_elevation)
        results["detected_azimuth"].append(detected_azimuth)

    return results

In [ ]:
class NotEnoughDataException(Exception):
    pass

# Given a path to a dataset (a directory containing many .exr files), it parallelizes calls to process_subset
# to detect the solar disk positions in all the images, and it tries to find a time offset in which these
# images were probably taken, given the difference between the expected sun disk positions and the detected
# sun disk positions from the images
def process_dataset(path):
    path = Path(path)

    def chunks(lst, n):
        """Yield successive n-sized chunks from lst."""
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    results = {
        "t": [],
        "lat": [],
        "lon": [],
        "detected_azimuth": [],
        "detected_elevation": [],
        "img_fname": [],
    }
    paths_exr = list(sorted(path.glob("*_hdr.exr")))#[::10]

    if len(paths_exr) < 50:
        raise NotEnoughDataException("There is less than 50 photos in the dataset, which is not enough to meaningfully fit the solar disk positions")

    print("Starting processing the dataset")

    # Parallel processing:
    pool_size = 8
    chunk_size = 20
    pool = mp.Pool(pool_size)
    paths_exr_chunked = list(chunks(paths_exr, chunk_size))
    with tqdm(total=len(paths_exr)) as progress_bar:
        for results_ in pool.imap_unordered(process_subset, paths_exr_chunked):
            progress_bar.update(len(results_["t"]))
            for key, value in results.items():
                value.extend(results_[key])

    # # Non-parallel processing:
    # results = process_subset(paths_exr)

    if np.count_nonzero(~np.isnan(results["detected_azimuth"])) < 20:
        raise NotEnoughDataException("Less than 20 solar disk positions were successfully detected, which is not enough to meaningfully fit the solar disk positions")

    # Sorting the results by time:
    print("Sorting the results by time")
    results_df = pd.DataFrame(results)
    results_df.sort_values(by="t", inplace=True)
    results = results_df.to_dict("list")

    expected_elevation_azimuth = np.array([
        compute_elevation_azimuth(lat, lon, dt) for lat, lon, dt in zip(results["lat"], results["lon"], results["t"])
    ])
    results["expected_elevation"] = expected_elevation_azimuth[:,0]
    results["expected_azimuth"] = expected_elevation_azimuth[:,1]

    # Finding the time offset:
    print("Finding the time offset")

    def find_time_offset():
        fig, axs = plt.subplots(nrows=2, tight_layout=True, figsize=(12,8))

        ax = axs[0]
        ax.set_title("Azimuth error and offset")
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M', tz=results["t"][0].tzinfo))
        ax = axs[1]
        ax.set_title("Elevation error and offset")
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M', tz=results["t"][0].tzinfo))

        best_error = np.inf
        best_time_offset_seconds = 0
        best_elevation_azimuth_offset = (0, 0)
        best_corrected_elevation_azimuth = []
        for time_offset_seconds in np.arange(-20*60, 20*60+15, 15, dtype=float):
            corrected_elevation_azimuth = np.array([
                compute_elevation_azimuth(lat, lon, dt + datetime.timedelta(seconds = time_offset_seconds)) for lat, lon, dt in zip(results["lat"], results["lon"], results["t"])
            ])
            error_elevation = np.array(results["detected_elevation"]) - corrected_elevation_azimuth[:,0]
            error_azimuth = np.array(results["detected_azimuth"]) - corrected_elevation_azimuth[:,1]
            elevation_azimuth_offset = (np.nanmedian(error_elevation), np.nanmedian(error_azimuth))
            corrected_elevation_azimuth[:,0] += elevation_azimuth_offset[0]
            corrected_elevation_azimuth[:,1] += elevation_azimuth_offset[1]
            error_elevation = np.array(results["detected_elevation"]) - corrected_elevation_azimuth[:,0]
            error_azimuth = np.array(results["detected_azimuth"]) - corrected_elevation_azimuth[:,1]
            error_elevation_0d = np.nanmedian(np.abs(error_elevation))
            error_azimuth_0d = np.nanmedian(np.abs(error_azimuth))
            error = error_elevation_0d + error_azimuth_0d
            ax = axs[0]
            ax.scatter(results["t"], error_azimuth, label=f"T={time_offset_seconds:.0f}s, Err={error_azimuth_0d:.3f}", s=3)
            ax = axs[1]
            ax.scatter(results["t"], error_elevation, label=f"T={time_offset_seconds:.0f}s, Err={error_elevation_0d:.3f}", s=3)
            if error < best_error:
                best_error = error
                best_time_offset_seconds = time_offset_seconds
                best_elevation_azimuth_offset = elevation_azimuth_offset
                best_corrected_elevation_azimuth = corrected_elevation_azimuth
        
        def plot_best_results():
            ax = axs[0]
            ax.plot(results["t"], np.array(results["detected_azimuth"]) - best_corrected_elevation_azimuth[:,1], label=f"Best", color="red")
            ax = axs[1]
            ax.plot(results["t"], np.array(results["detected_elevation"]) - best_corrected_elevation_azimuth[:,0], label=f"Best", color="red")
        plot_best_results()

        def plot_zeros():
            ax = axs[0]
            ax.axhline(y = 0, color="black", linestyle="--")
            ax = axs[1]
            ax.axhline(y = 0, color="black", linestyle="--")
        plot_zeros()

        for ax in axs:
            ax.set_xlim(sorted(results["t"])[0], sorted(results["t"])[-1])
            # ax.legend()
            ax.grid()

        return best_time_offset_seconds, best_elevation_azimuth_offset, best_corrected_elevation_azimuth

    time_offset, elevation_azimuth_offset, corrected_elevation_azimuth = find_time_offset()

    print("Corrected time offset:", time_offset)

    results["corrected_elevation"] = corrected_elevation_azimuth[:,0]
    results["corrected_azimuth"] = corrected_elevation_azimuth[:,1]

    return results

### The actual "main" cell that runs the script for the selected dataset paths:

In [ ]:
shooting_names = '''2019-05-10_1755_podbaba
2019-05-17_1946_hodejovice
2019-05-18_1357_hodejovice
2019-05-24_1451_divoka_sarka
2019-06-04_1010_ronan
2019-06-04_1117_ronan
2019-06-04_1659_ronan
#2019-06-05_1215_ronan - the camera moves while shooting
2019-06-05_1700_ronan
2019-06-06_1158_ronan
2019-06-06_1257_ronan
2019-06-07_1306_hammerschmiede
2019-06-08_1545_quarry
2019-06-10_1235_hammerschmiede
2019-06-12_1744_stromovka_cgbbq
2019-06-16_1906_prague_chodska_21_rooftop
2019-06-18_0523_prague_chodska_21_rooftop
#2019-06-20_0454_prague_chodska_21_rooftop - exr vs. raw mismatch!
2019-06-20_0851_prague_chodska_21_rooftop
2019-06-20_1151_prague_chodska_21_rooftop
#2019-06-21_0825_prague_chodska_21_rooftop - some mixup? the shoot is from the morning, but the Sun is going down???
2019-06-22_1859_farm_vystice
2019-06-23_0943_farm_vystice
2019-06-23_1939_prague_chodska_21_rooftop
#2019-06-24_0504_prague_chodska_21_rooftop - exr vs. raw mismatch
2019-06-24_0827_prague_chodska_21_rooftop
2019-06-25_1952_prague_chodska_21_rooftop
2019-06-26_0848_prague_chodska_21_rooftop
2019-06-26_1311_prague_chodska_21_rooftop
2019-06-26_1855_prague_chodska_21_rooftop
2019-06-29_1113_divoka_sarka
2019-07-04_1749_lake_lhota
2019-07-05_0812_prague_chodska_21_rooftop
#2019-07-07_2132_prague_chodska_21_rooftop - night, fits to the moon instead of the sun
2019-07-13_1239_prague_chodska_21_rooftop
2019-07-13_1524_prague_chodska_21_rooftop
2019-07-13_1905_prague_chodska_21_rooftop
2019-07-14_1054_prague_chodska_21_rooftop
2019-07-14_1317_prague_chodska_21_rooftop
2019-07-14_1418_prague_chodska_21_rooftop
2019-07-14_1603_prague_chodska_21_rooftop
2019-07-14_1618_prague_chodska_21_rooftop
2019-07-15_0844_prague_chodska_21_rooftop
2019-07-15_1929_prague_chodska_21_rooftop
2019-07-16_0845_prague_chodska_21_rooftop
2019-07-16_1221_prague_chodska_21_rooftop
2019-07-17_0547_prague_chodska_21_rooftop
2019-07-17_0856_prague_chodska_21_rooftop
2019-07-17_1732_prague_chodska_21_rooftop
2019-07-17_1957_prague_chodska_21_rooftop
2019-08-05_0841_santa_cruz_villa_nuova
#2019-08-05_1716_santa_cruz_villa_nuova - poor results
2019-08-05_2013_santa_cruz_villa_nuova
2019-08-06_0933_santa_cruz_villa_nuova
2019-08-06_1657_santa_cruz_villa_nuova
2019-08-08_1048_santa_cruz_villa_nuova
2019-08-09_0800_santa_cruz_villa_nuova
2019-08-09_1800_santa_cruz_villa_nuova
2019-08-10_1000_santa_cruz_villa_nuova
2019-08-10_1959_santa_cruz_villa_nuova
2019-08-16_1130_burbank_1200_riverside
2019-08-16_1824_burbank_1200_riverside'''.split('\n')

for i, shooting_name in enumerate(shooting_names):
    if shooting_name[0] == "#":
        print(f"{i:03d}/{len(shooting_names):03d} Skipping dataset", shooting_name)
        continue

    try:
        path = Path("/projects/SkyGAN/clouds_fisheye/processed/") / shooting_name / "1K_EXR"
        output_path = Path("/projects/SkyGAN/clouds_fisheye/solar_disk_fitting/") / shooting_name
        output_path.mkdir(parents=True, exist_ok=True)

        print(f"{i:03d}/{len(shooting_names):03d} Processing dataset", path)

        results = process_dataset(path)
        plt.savefig(str(output_path / "time_offset_fitting.png"))
        plt.show()

        fig, axs = plt.subplots(nrows=2, tight_layout=True, figsize=(12,8))
        ax = axs[0]
        ax.set_title("Azimuth")
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M', tz=results["t"][0].tzinfo))
        ax.plot(results["t"], results["expected_azimuth"], label="Expected (from date, time, and location)", zorder=1, linewidth=3)
        ax.scatter(results["t"], results["detected_azimuth"], label="Detected (from image)", zorder=3, s=10, color="C1")
        ax.plot(results["t"], results["corrected_azimuth"], label="Corrected", zorder=2, linewidth=3, color="C2")
        # ax.set_ylim(60,70)
        # y_fit = fit_curve([dt.timestamp() for dt in results["t"]], results["detected_azimuth"])
        # ax.plot(results["t"], y_fit, color="C1")
        ax.legend()
        ax.grid()
        ax = axs[1]
        ax.set_title("Elevation")
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M', tz=results["t"][0].tzinfo))
        ax.plot(results["t"], results["expected_elevation"], label="Expected (from date, time, and location)", zorder=1, linewidth=3)
        ax.scatter(results["t"], results["detected_elevation"], label="Detected (from image)", zorder=3, s=10, color="C1")
        ax.plot(results["t"], results["corrected_elevation"], label="Corrected", zorder=2, linewidth=3, color="C2")
        ax.legend()
        ax.grid()

        plt.savefig(str(output_path / "solar_disk_fitting.png"))
        plt.show()

        np.savez_compressed(str(output_path / "fitting_results.npz"), **results)
    except NotEnoughDataException as e:
        print("!!! !!! !!!")
        print(f"Skipping dataset {shooting_name}")
        print(e)
        print("!!! !!! !!!")
    except Exception as e:
        import traceback
        print("!!! !!! !!!")
        print(f"Exception caught, dataset {shooting_name} could not be processed")
        # print(e)
        traceback.print_exc()
        print("!!! !!! !!!")
